# **Step 1.** Install Python Packages

In [ ]:
!pip install -U xformers --index-url https://download.pytorch.org/whl/cu121
!pip install --no-deps packaging ninja einops flash-attn trl peft accelerate bitsandbytes
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 46.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Using cached ninja-1.11.1.3-py3-none-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (422 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 8.9 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.7.3-cp311-cp311-linux_x86_64.whl size=191363917 sha256=b1243e9b86687348a5ab03a073abacdf8e3d5e9e4b7e5326a183f47348c5dfba
  Stored in directory: /root/.cache/pip/wheels/d0/a3/f9/48d2706cb2eac05ec0dc144bf6954fe47bb3c2cd0de280765e
Successfully built flash-attn
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-7timzd9y/unsloth_94b4d1bdc93c4ea7aaca766f5e6614eb
  Running command git clone --filter=blob:none --quiet h

# **Step 2.** Import Python Packages

In [ ]:
import torch
import os
import json
import pandas as pd
from datasets import Dataset, DatasetDict
from datasets import load_dataset
from huggingface_hub import notebook_login
from transformers import TrainingArguments
from trl import SFTTrainer
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


# **Step 3.** Login to Your Hugging Face with hf_token. (write access token)

In [ ]:
notebook_login()

In [ ]:
'''from datasets import load_dataset

ds = load_dataset("engineersaloni159/INS-indian-legal-dataset")
data=ds['train']
# Print the column names to check what they are
#print(data.columns())

# View the first few rows of the data to understand its structure
# Display the column names of the dataset
print(data.column_names)
'''


['judgement', 'summary']


# **Step 4.** Convert your JSON dataset to Llama3 finetuning format


In [ ]:
huggingface_user = "Dominic2106"
dataset_name = "Legal_Advisor-finetune"

class Llama3InstructDataset:
    def __init__(self, data):
        self.data = data
        self.prompts = []
        self.create_prompts()

    def create_prompt(self, row):
        prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>{row['instruction']}<|eot_id|><|start_header_id|>user<|end_header_id|>{row['input']}<|eot_id|><|start_header_id|>assistant<|end_header_id|>{row['output']}<|eot_id|>"""
        #prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>{'You are a legal advisor chatbot'}<|eot_id|><|start_header_id|>user<|end_header_id|>{row['judgement']}<|eot_id|><|start_header_id|>assistant<|end_header_id|>{row['summary']}<|eot_id|>"""

        return prompt

    def create_prompts(self):
        for row in self.data:
            prompt = self.create_prompt(row)
            self.prompts.append(prompt)

    def get_dataset(self):
        df = pd.DataFrame({'prompt': self.prompts})
        return df

def create_dataset_hf(dataset):
    dataset.reset_index(drop=True, inplace=True)
    return DatasetDict({"train": Dataset.from_pandas(dataset)})

if __name__ == "__main__":
    # Fetching dataset using load_dataset
    ds = load_dataset("Sahil2507/Indian_Legal_Dataset")

    # Fetching the dataset's 'train' split
    data = ds['train']

    dataset = Llama3InstructDataset(data)
    df = dataset.get_dataset()


    processed_data_path = 'processed_data'
    os.makedirs(processed_data_path, exist_ok=True)

    llama3_dataset = create_dataset_hf(df)
    llama3_dataset.save_to_disk(os.path.join(processed_data_path, "llama3_dataset"))
    llama3_dataset.push_to_hub(f"{huggingface_user}/{dataset_name}")

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

legal_DataSet.jsonl:   0%|          | 0.00/16.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4394 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/4394 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

# **Step 5.** LoRa Finetuning Configurations
- "finetuned_model" sets your models name on HF
- "num_train_epochs" sets the number of epochs for training

    (epoch = 1 pass through your entire dataset)

In [ ]:
# Defining the configuration for the base model, LoRA and training
config = {
    "hugging_face_username":huggingface_user,
    "model_config": {
        "base_model":"unsloth/llama-3-8b-Instruct-bnb-4bit", # The base model
        "finetuned_model":"llama-3-Mental-Therapy-FineTune", # The finetuned model
        "max_seq_length": 2048, # The maximum sequence length
        "dtype":torch.float16, # The data type
        "load_in_4bit": True, # Load the model in 4-bit
    },
    "lora_config": {
      "r": 16, # The number of LoRA layers 8, 16, 32, 64
      "target_modules": ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"], # The target modules
      "lora_alpha":16, # The alpha value for LoRA
      "lora_dropout":0, # The dropout value for LoRA
      "bias":"none", # The bias for LoRA
      "use_gradient_checkpointing":True, # Use gradient checkpointing
      "use_rslora":False, # Use RSLora
      "use_dora":False, # Use DoRa
      "loftq_config":None # The LoFTQ configuration
    },
    "training_dataset":{
        "name":f"{huggingface_user}/{dataset_name}", # The dataset name(huggingface/datasets)
        "split":"train", # The dataset split
        "input_field":"prompt", # The input field
    },
    "training_config": {
        "per_device_train_batch_size": 2, # The batch size
        "gradient_accumulation_steps": 4, # The gradient accumulation steps
        "warmup_steps": 5, # The warmup steps
        "max_steps":50, # The maximum steps (0 if the epochs are defined)
        "num_train_epochs": 0, # The number of training epochs(0 if the maximum steps are defined)
        "learning_rate": 2e-4, # The learning rate
        "fp16": not torch.cuda.is_bf16_supported(),  # The fp16
        "bf16": torch.cuda.is_bf16_supported(), # The bf16
        "logging_steps": 1, # The logging steps
        "optim" :"adamw_8bit", # The optimizer
        "weight_decay" : 0.01,  # The weight decay
        "lr_scheduler_type": "linear", # The learning rate scheduler
        "seed" : 42, # The seed
        "output_dir" : "outputs", # The output directory
    }
}

# **Step 6.** Load Llama3-8B, QLoRA & Trainer Model

In [ ]:
# Loading the model and the tokinizer for the model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = config.get("model_config").get("base_model"),
    max_seq_length = config.get("model_config").get("max_seq_length"),
    dtype = config.get("model_config").get("dtype"),
    load_in_4bit = config.get("model_config").get("load_in_4bit"),
)

# Setup for QLoRA/LoRA peft of the base model
model = FastLanguageModel.get_peft_model(
    model,
    r = config.get("lora_config").get("r"),
    target_modules = config.get("lora_config").get("target_modules"),
    lora_alpha = config.get("lora_config").get("lora_alpha"),
    lora_dropout = config.get("lora_config").get("lora_dropout"),
    bias = config.get("lora_config").get("bias"),
    use_gradient_checkpointing = config.get("lora_config").get("use_gradient_checkpointing"),
    random_state = 42,
    use_rslora = config.get("lora_config").get("use_rslora"),
    use_dora = config.get("lora_config").get("use_dora"),
    loftq_config = config.get("lora_config").get("loftq_config"),
)

# Loading the training dataset
dataset_train = load_dataset(config.get("training_dataset").get("name"), split = config.get("training_dataset").get("split"))

# Setting up the trainer for the model
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_train,
    dataset_text_field = config.get("training_dataset").get("input_field"),
    max_seq_length = config.get("model_config").get("max_seq_length"),
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = config.get("training_config").get("per_device_train_batch_size"),
        gradient_accumulation_steps = config.get("training_config").get("gradient_accumulation_steps"),
        warmup_steps = config.get("training_config").get("warmup_steps"),
        max_steps = config.get("training_config").get("max_steps"),
        num_train_epochs= config.get("training_config").get("num_train_epochs"),
        learning_rate = config.get("training_config").get("learning_rate"),
        fp16 = config.get("training_config").get("fp16"),
        bf16 = config.get("training_config").get("bf16"),
        logging_steps = config.get("training_config").get("logging_steps"),
        optim = config.get("training_config").get("optim"),
        weight_decay = config.get("training_config").get("weight_decay"),
        lr_scheduler_type = config.get("training_config").get("lr_scheduler_type"),
        seed = 42,
        output_dir = config.get("training_config").get("output_dir"),
    ),
)

==((====))==  Unsloth 2025.1.7: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

...

Unsloth 2025.1.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


README.md:   0%|          | 0.00/276 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.57M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4394 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/4394 [00:00<?, ? examples/s]

# **Step 7.** Train Your Finetuned Model

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 4,394 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 50
 "-____-"     Number of trainable parameters = 41,943,040
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
1,1.981600
2,2.535700
3,1.747900
4,1.701300
5,1.645100
6,1.316700
7,1.507600
8,1.269500
9,1.303300
10,1.223600


# **Step 8.** Save Trainer Stats

In [ ]:
with open("trainer_stats.json", "w") as f:
    json.dump(trainer_stats, f, indent=4)

# **Step 9.** Save Finetuned Model & Push to HF Hub

In [ ]:
#model.save_pretrained_gguf(config.get("model_config").get("finetuned_model"), tokenizer, quantization_method = "q4_k_m")
model.push_to_hub_gguf(config.get("model_config").get("finetuned_model"), tokenizer, quantization_method = "q4_k_m")

# **Step 10.** Test your pretrained model in Colab

In [ ]:
# Loading the fine-tuned model and the tokenizer for inference
model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = config.get("model_config").get("finetuned_model"),
        max_seq_length = config.get("model_config").get("max_seq_length"),
        dtype = config.get("model_config").get("dtype"),
        load_in_4bit = config.get("model_config").get("load_in_4bit"),
    )

# Using FastLanguageModel for fast inference
FastLanguageModel.for_inference(model)

system_prompt = f"You are an AI task automator. You will take a users prompt and use first principle reasoning to break the prompt into tasks that you must accomplish within another chat. RESPOND TO THIS MESSAGE ONLY WITH A PYTHON FORMATTED LIST OF TASKS THAT YOU MUST COMPLETE TO TRUTHFULLY AND INTELLIGENTLY ACCOMPLISH THE USERS REQUEST. ASSUME YOU CAN SEARCH THE WEB, WRITE CODE, RUN CODE, DEBUG CODE, AND AUTOMATE ANYTHING ON THE USERS COMPUTER TO ACCOMPLISH THE PROMPT. CORRECT RESPONSE FORMAT: ['task 1', 'task 2', 'task 3']"

# Tokenizing the input and generating the output
prompt = input('TYPE PROMPT TO LLAMA3: ')
inputs = tokenizer(
[
    f"<|start_header_id|>system<|end_header_id|>{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>{prompt}<|end_header_id|>"
], return_tensors = "pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
tokenizer.batch_decode(outputs, skip_special_tokens = True)

==((====))==  Unsloth 2025.1.7: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

TYPE PROMPT TO LLAMA3: hey


["systemYou are an AI task automator. You will take a users prompt and use first principle reasoning to break the prompt into tasks that you must accomplish within another chat. RESPOND TO THIS MESSAGE ONLY WITH A PYTHON FORMATTED LIST OF TASKS THAT YOU MUST COMPLETE TO TRUTHFULLY AND INTELLIGENTLY ACCOMPLISH THE USERS REQUEST. ASSUME YOU CAN SEARCH THE WEB, WRITE CODE, RUN CODE, DEBUG CODE, AND AUTOMATE ANYTHING ON THE USERS COMPUTER TO ACCOMPLISH THE PROMPT. CORRECT RESPONSE FORMAT: ['task 1', 'task 2', 'task 3']userheyassistant\n\nI'm happy to help! However, I need a more specific prompt to understand what you need assistance with. Please provide a clear and concise request, and I'll break it down into tasks for you."]

In [ ]:
!zip -r llama_model.zip /content/llama-3-Mental-Therapy-FineTune


  adding: content/llama-3-Mental-Therapy-FineTune/ (stored 0%)
  adding: content/llama-3-Mental-Therapy-FineTune/pytorch_model-00004-of-00004.bin (deflated 22%)
  adding: content/llama-3-Mental-Therapy-FineTune/unsloth.Q4_K_M.gguf (deflated 2%)
  adding: content/llama-3-Mental-Therapy-FineTune/special_tokens_map.json (deflated 70%)
  adding: content/llama-3-Mental-Therapy-FineTune/pytorch_model-00001-of-00004.bin (deflated 12%)
  adding: content/llama-3-Mental-Therapy-FineTune/tokenizer_config.json (deflated 96%)
  adding: content/llama-3-Mental-Therapy-FineTune/config.json (deflated 48%)
  adding: content/llama-3-Mental-Therapy-FineTune/pytorch_model-00003-of-00004.bin (deflated 9%)
  adding: content/llama-3-Mental-Therapy-FineTune/pytorch_model.bin.index.json (deflated 95%)
  adding: content/llama-3-Mental-Therapy-FineTune/generation_config.json (deflated 35%)
  adding: content/llama-3-Mental-Therapy-FineTune/tokenizer.json (deflated 85%)
  adding: content/llama-3-Mental-Therapy-Fine